In [1]:
import numpy as np

from scipy import stats, signal
from sklearn import linear_model
import random


# %qtconsole
%matplotlib qt5
%matplotlib auto
import matplotlib.pyplot as plt


Using matplotlib backend: Qt5Agg


In [5]:
%qtconsole

## Simple models of speed vs. SF vs SL

In [101]:
# null model of steady walking monopod

# CONSTANT VELOCITY, CONSTANT FREQUENCY

n_strides = 100 # how many strides want to model?

v = 10 # mm/s
freq = 5 # Hz
dur = 1/freq # in seconds

placement_error_factor = 0.6
placement_error = np.random.sample(size=n_strides)*placement_error_factor-placement_error_factor/2

TD_times = np.arange(0,n_strides)*dur
body_positions = np.arange(0,n_strides)*dur*v
leg_positions = body_positions+placement_error

stride_lens = np.diff(leg_positions)
stride_speeds = np.diff(body_positions)/np.diff(TD_times)

plt.close('all')
plt.figure()
plt.plot(stride_lens, stride_speeds, '.k', label = 'actual')
plt.plot(stride_lens, stride_lens*freq, '.b', label = 'SL*SF')
plt.ylim([0,15])
plt.xlabel('stride length (mm)')
plt.ylabel('stride speed (mm/s)')
plt.legend()
plt.title('speed = %i mm/s, freq = %i Hz'%(v, freq))

# for nn in np.arange(0,n_strides):
#     print('TD %i: time = %0.1f s, body position = %0.2f, leg position = %0.2f'%(nn, nn*dur, nn*dur*v, nn*dur*v+placement_error[nn]))

Text(0.5,1,'speed = 10 mm/s, freq = 5 Hz')

In [1505]:
n_strides = 100 # how many strides want to model?

# CONSTANT VELOCITY, CONSTANT LENGTH

v = 10 # mm/s
SL = 2 # mm


placement_error_factor = 0.6
placement_error = np.random.sample(size=n_strides)*placement_error_factor-placement_error_factor/2

leg_positions = np.arange(0,n_strides)*SL
body_positions = leg_positions - placement_error
TD_times = body_positions/v
stride_lengths = np.diff(leg_positions)
dur = np.diff(TD_times)
stride_freqs = 1/dur
stride_speeds = np.diff(body_positions)/dur

# print(dur, leg_positions)

plt.close('all')
plt.figure()
plt.plot(stride_freqs, stride_speeds, '.k', label = 'actual')
plt.plot(stride_freqs, stride_lengths/dur, '.b', label = 'SL*SF')
plt.xlabel('stride freq (Hz)')
plt.ylabel('stride speed (mm/s)')
plt.legend()
plt.title('speed = %i mm/s, length = %i mm'%(v, SL))

Text(0.5,1,'speed = 10 mm/s, length = 2 mm')

In [1537]:
# CHANGING VELOCITY, CONSTANT FREQUENCY

n_strides = 1000 # how many strides want to model?

v = 10 # mm/s
freq = 5 # Hz
dur = 1/freq # in seconds

placement_error_factor = 0.6
placement_error = np.random.sample(size=n_strides)*placement_error_factor-placement_error_factor/2

v_error_factor = 20 
v_error = np.random.sample(size=n_strides)*v_error_factor-v_error_factor/2
vs = v+v_error

TD_times = np.arange(0,n_strides)*dur
body_positions = np.cumsum(vs*dur)
leg_positions = body_positions+placement_error

stride_lens = np.diff(leg_positions)
stride_speeds = np.diff(body_positions)/np.diff(TD_times)

plt.close('all')
plt.figure()
plt.plot(stride_lens, vs[1:], '.k', label = 'actual')
# plt.plot(stride_lens, stride_speeds, '+k', label = 'actual')
plt.plot(stride_lens, stride_lens*freq, '.b', label = 'SL*SF')
# plt.ylim([0,15])
plt.xlabel('stride length (mm)')
plt.ylabel('stride speed (mm/s)')
plt.legend()
plt.title('speed = %i mm/s +/- %i, freq = %i Hz'%(v, v_error_factor/2, freq))


Text(0.5,1,'speed = 10 mm/s +/- 10, freq = 5 Hz')

In [102]:
n_strides = 1000 # how many strides want to model?

# CHANGING VELOCITY, CONSTANT LENGTH

v = 10 # mm/s
SL = 2 # mm

v_error_factor = 20 
v_error = np.random.sample(size=n_strides)*v_error_factor-v_error_factor/2
vs = v+v_error

placement_error_factor = 0.6
placement_error = np.random.sample(size=n_strides)*placement_error_factor-placement_error_factor/2

leg_positions = np.arange(0,n_strides)*SL
body_positions = leg_positions - placement_error
TD_times = np.cumsum(np.insert(np.diff(body_positions)/vs[:-1],0,0))
stride_lengths = np.diff(leg_positions)
dur = np.diff(TD_times)
stride_freqs = 1/dur
stride_speeds = np.diff(body_positions)/dur

# # print(dur, leg_positions)

plt.close('all')
plt.figure()
plt.plot(stride_freqs, stride_speeds, '.k', label = 'actual')
# plt.plot(stride_freqs, vs[:-1], '+k', label = 'actual')
plt.plot(stride_freqs, stride_lengths/dur, '.b', label = 'SL*SF')
plt.xlabel('stride freq (Hz)')
plt.ylabel('stride speed (mm/s)')
plt.legend()
plt.title('speed = %i mm/s +/- %i, length = %i mm'%(v, v_error_factor/2, SL))

Text(0.5,1,'speed = 10 mm/s +/- 10, length = 2 mm')

In [103]:
# CONTINUOUSLY CHANGING VELOCITY, CONSTANT FREQUENCY

n_strides = 1000 # how many strides want to model?

v = 10 # mm/s
freq = 5 # Hz
dur = 1/freq # in seconds

placement_error_factor = 0.6
placement_error = np.random.sample(size=n_strides)*placement_error_factor-placement_error_factor/2

resolution = 100
t = np.arange(0,n_strides/freq, 1/resolution)
w=2*freq
v_error_factor = 10 
vs =  v+v_error_factor/2*np.sin(t*w*2*np.pi)+ 3*np.sin(t*w/10*2*np.pi) + 1*np.sin(t*w/30*2*np.pi)


TD_times = np.arange(0,n_strides)*dur
TD_idcs = (TD_times*resolution).astype(int)
body_positions = np.insert(np.cumsum(vs/resolution),0,0)[TD_idcs]
leg_positions = body_positions+placement_error

stride_lens = np.diff(leg_positions)
stride_speeds = np.diff(body_positions)/np.diff(TD_times)
stride_vs_ave = (np.cumsum(vs)[TD_idcs][1:]-np.cumsum(vs)[TD_idcs][:-1])/(dur*resolution)
SL_by_SF = stride_lens*freq

# linear regression of SL*SF
from sklearn import linear_model
slope, intercept, r_val, p_val, std_err = stats.linregress( stride_lens, SL_by_SF)


plt.close('all')

# # plot time varying speed
plt.figure()
plt.plot(t, vs, '-')
plt.plot([TD_times[:-1],TD_times[1:]], [stride_vs_ave, stride_vs_ave], '-r')
plt.xlim([0,10])
plt.ylabel('velocity (mm/s)')
plt.xlabel('time (s)')

plt.figure()
# plt.plot(stride_lens, vs[1:], '.k', label = 'actual')
# plt.plot(stride_lens, stride_speeds, '+k', label = 'actual')
plt.plot(stride_lens, stride_vs_ave, '+k', label = 'ave of continous v')
plt.plot(stride_lens, stride_lens*freq, '.b', label = 'SL*SF')
# plt.ylim([0,15])
plt.xlabel('stride length (mm)')
plt.ylabel('stride speed (mm/s)')
plt.text(plt.gca().get_xlim()[1]-1, plt.gca().get_ylim()[0]+1, 'y = %0.2f x + %0.2f'%(slope, intercept), color = 'b')
plt.legend()
plt.title('speed = %i mm/s +/- %i, freq = %i Hz'%(v, v_error_factor/2, freq))

Text(0.5,1,'speed = 10 mm/s +/- 5, freq = 5 Hz')

In [25]:
# CONTINUOUSLY CHANGING VELOCITY, CONSTANT LENGTH

n_strides = 1000 # how many strides want to model?

v = 10 # mm/s
SL = 2 # mm

placement_error_factor = 0.6
placement_error = np.random.sample(size=n_strides)*placement_error_factor-placement_error_factor/2

resolution = 1000
t = np.arange(0,n_strides/freq, 1/resolution)
w=2*freq
v_error_factor = 10 
vs =  v+v_error_factor/2*np.sin(t*w*2*np.pi)+ 3*np.sin(t*w/10*2*np.pi) + 1*np.sin(t*w/30*2*np.pi)


leg_positions = np.arange(0,n_strides)*SL
body_positions = leg_positions - placement_error
TD_idcs = np.array([np.argmin(np.abs(bp-np.insert(np.cumsum(vs/resolution),0,0))) for bp in body_positions])
TD_times = TD_idcs/resolution # in seconds
stride_lengths = np.diff(leg_positions)
dur = np.diff(TD_times)
stride_freqs = 1/dur
stride_speeds = np.diff(body_positions)/dur
stride_vs_ave = (np.cumsum(vs)[TD_idcs][1:]-np.cumsum(vs)[TD_idcs][:-1])/(dur*resolution)


# # linear regression of SL*SF
# from sklearn import linear_model
# slope, intercept, r_val, p_val, std_err = stats.linregress( stride_lens, SL_by_SF)


plt.close('all')

# # plot time varying speed
plt.figure()
plt.plot(t, vs, '-')
plt.plot([TD_times[:-1],TD_times[1:]], [stride_vs_ave, stride_vs_ave], '-r')
plt.xlim([0,10])
plt.ylabel('velocity (mm/s)')
plt.xlabel('time (s)')

plt.figure()
# plt.plot(stride_freqs, stride_speeds, '.k', label = 'actual')
plt.plot(stride_freqs, stride_vs_ave, '+k', label = 'ave of instant. vel')
plt.plot(stride_freqs, stride_lengths/dur, '.b', label = 'SL*SF')
plt.xlabel('stride freq (Hz)')
plt.ylabel('stride speed (mm/s)')
# plt.text(plt.gca().get_xlim()[1]-1, plt.gca().get_ylim()[0]+1, 'y = %0.2f x + %0.2f'%(slope, intercept), color = 'b')
plt.legend()
plt.title('speed = %i mm/s +/- %i, length = %i mm'%(v, v_error_factor/2, SL))

Text(0.5,1,'speed = 10 mm/s +/- 5, length = 2 mm')

In [104]:
# CONTINUOUSLY CHANGING VELOCITY, VARYING LENGTH AND FREQUENCY! NO BODY OFFSET

n_strides = 1000 # how many strides want to model?

v = 10 # mm/s
SL = 2 # mm

placement_error_factor = 0.6
placement_error = np.random.sample(size=n_strides)*placement_error_factor-placement_error_factor/2
SLs = SL+placement_error

resolution = 1000
t = np.arange(0,n_strides*SL/v*1.5, 1/resolution)
w=2*freq
v_error_factor = 10 
vs =  v+v_error_factor/2*np.sin(t*w*2*np.pi)+ 3*np.sin(t*w/10*2*np.pi) + 1*np.sin(t*w/30*2*np.pi)


leg_positions = np.insert(np.cumsum(SLs),0,0)
body_positions = leg_positions
TD_idcs = np.array([np.argmin(np.abs(bp-np.insert(np.cumsum(vs/resolution),0,0))) for bp in body_positions])
TD_times = TD_idcs/resolution # in seconds
stride_lengths = np.diff(leg_positions)
dur = np.diff(TD_times)
stride_freqs = 1/dur
stride_speeds = np.diff(body_positions)/dur
stride_vs_ave = (np.insert(np.cumsum(vs),0,0)[TD_idcs][1:]-np.insert(np.cumsum(vs),0,0)[TD_idcs][:-1])/(dur*resolution)
SL_by_SF = stride_lengths/dur


# linear regression of SL*SF
from sklearn import linear_model
slope, intercept, r_val, p_val, std_err = stats.linregress( stride_freqs[np.isfinite(stride_freqs)], SL_by_SF[np.isfinite(SL_by_SF)])


plt.close('all')

# # plot time varying speed
plt.figure()
plt.plot(t, vs, '-')
plt.plot([TD_times[:-1],TD_times[1:]], [stride_vs_ave, stride_vs_ave], '-r')
plt.xlim([0,10])
plt.ylabel('velocity (mm/s)')
plt.xlabel('time (s)')

plt.figure()
# plt.plot(stride_freqs, stride_speeds, '.k', label = 'actual')
plt.plot(stride_freqs, stride_vs_ave, '+k', label = 'ave of instant. vel')
plt.plot(stride_freqs, SL_by_SF, '.b', label = 'SL*SF')
plt.xlabel('stride freq (Hz)')
plt.ylabel('stride speed (mm/s)')
plt.text(plt.gca().get_xlim()[1]-2, plt.gca().get_ylim()[0]+1, 'y = %0.2f x + %0.2f'%(slope, intercept), color = 'b')
plt.plot(plt.gca().get_xlim(), slope*np.array(plt.gca().get_xlim())+intercept, '-b')
plt.legend()
plt.title('speed = %i mm/s +/- %i, length = %i mm'%(v, v_error_factor/2, SL))

plt.figure()
# plt.plot(stride_lens, vs[1:], '.k', label = 'actual')
# plt.plot(stride_lens, stride_speeds, '+k', label = 'actual')
plt.plot(stride_lengths, stride_vs_ave, '+k', label = 'ave of continous v')
plt.plot(stride_lengths, stride_lengths*stride_freqs, '.b', label = 'SL*SF')
# plt.ylim([0,15])
plt.xlabel('stride length (mm)')
plt.ylabel('stride speed (mm/s)')
plt.legend()
plt.title('speed = %i +/- %i mm/s  ---  freq = %i Hz'%(v, v_error_factor/2, freq))

Text(0.5,1,'speed = 10 +/- 5 mm/s  ---  freq = 5 Hz')

In [106]:
# CONTINUOUSLY CHANGING VELOCITY, VARYING LENGTH AND FREQUENCY! WITH BODY OFFSET

n_strides = 1000 # how many strides want to model?

v = 10 # mm/s
SL = 2 # mm

SL_error_factor = SL/8
SL_error = np.random.sample(size=n_strides)*placement_error_factor-placement_error_factor/2
SLs = SL+SL_error

placement_error_factor = SL/2
placement_error = np.random.sample(size=n_strides)*placement_error_factor-placement_error_factor/2

resolution = 1000
t = np.arange(0,n_strides*SL/v*1.5, 1/resolution)
w=2*freq
v_error_factor = 10 
vs =  v+v_error_factor/2*np.sin(t*w*2*np.pi)+ 3*np.sin(t*w/10*2*np.pi) + 1*np.sin(t*w/30*2*np.pi)


leg_positions = np.insert(np.cumsum(SLs),0,0)[:-1]
body_positions = leg_positions-placement_error
TD_idcs = np.array([np.argmin(np.abs(bp-np.insert(np.cumsum(vs/resolution),0,0))) for bp in body_positions])
TD_times = TD_idcs/resolution # in seconds
stride_lengths = np.diff(leg_positions)
dur = np.diff(TD_times)
stride_freqs = 1/dur
stride_speeds = np.diff(body_positions)/dur
stride_vs_ave = (np.insert(np.cumsum(vs),0,0)[TD_idcs][1:]-np.insert(np.cumsum(vs),0,0)[TD_idcs][:-1])/(dur*resolution)
SL_by_SF = stride_lengths/dur


# linear regression of SL*SF
from sklearn import linear_model
slope, intercept, r_val, p_val, std_err = stats.linregress( stride_freqs[np.isfinite(stride_freqs)], SL_by_SF[np.isfinite(SL_by_SF)])


plt.close('all')

# # plot time varying speed
plt.figure()
plt.plot(t, vs, '-k', alpha = 0.7)
plt.plot([TD_times[:-1],TD_times[1:]], [stride_vs_ave, stride_vs_ave], '-b')
plt.xlim([0,10])
plt.ylabel('velocity (mm/s)')
plt.xlabel('time (s)')

plt.figure()
plt.plot(stride_freqs, stride_speeds, '+r', label = 'net body displacement')
plt.plot(stride_freqs, stride_vs_ave, '+k', label = 'ave of instant. vel')
plt.plot(stride_freqs, SL_by_SF, '.b', label = 'SL*SF')
plt.xlabel('stride freq (Hz)')
plt.ylabel('stride speed (mm/s)')
plt.text(plt.gca().get_xlim()[1]-2, plt.gca().get_ylim()[0]+1, 'y = %0.2f x + %0.2f'%(slope, intercept), color = 'b')
plt.plot(plt.gca().get_xlim(), slope*np.array(plt.gca().get_xlim())+intercept, '-b')
plt.legend()
plt.title('speed = %i +/- %i mm/s  ---  length = %i +/- %0.1f mm'%(v, v_error_factor/2, SL, SL_error_factor/2))

plt.figure()
plt.plot(stride_lengths, stride_speeds, '+r', label = 'net body displacement')
plt.plot(stride_lengths, stride_vs_ave, '+k', label = 'ave of continous v')
plt.plot(stride_lengths, stride_lengths*stride_freqs, '.b', label = 'SL*SF')
plt.xlabel('stride length (mm)')
plt.ylabel('stride speed (mm/s)')
plt.legend()
plt.title('speed = %i +/- %i mm/s  ---  length = %i +/- %0.1f mm'%(v, v_error_factor/2, SL, SL_error_factor/2))

Text(0.5,1,'speed = 10 +/- 5 mm/s  ---  length = 2 +/- 0.1 mm')

In [114]:
# make disrupted stride diagrams
n_strides = 2
freq = 10
resolution = 100
t = np.arange(0,n_strides/freq, 1/resolution/freq)
w=freq
position = 2*np.sin(t*w*2*np.pi)

plt.close('all')
plt.figure()
plt.plot(position, t, '-k')
plt.gca().invert_yaxis()

# v_error_factor = 10 
# vs =  v+v_error_factor/2*np.sin(t*w*2*np.pi)+ 3*np.sin(t*w/10*2*np.pi) + 1*np.sin(t*w/30*2*np.pi)